In [1]:
%%capture
%cd scripts

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook

# Imports

In [3]:
from itertools import product
import os

import graph_tool.all as gt
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import hypergeom, pearsonr
import seaborn as sns
from sklearn.cluster import KMeans

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')

# Style
sns.set_theme(context='talk', style='white', palette='Set2')
plt.rcParams.update({
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 22,
    'axes.titlesize': 'medium',
    'axes.labelsize': 'large',
    'xtick.labelsize': 'medium',
    'ytick.labelsize': 'medium',
    'legend.fontsize': 'medium',
    'legend.title_fontsize': 'medium',
    'figure.titlesize': 'x-large',
})

# Figure transparency
# matplotlib.rcParams['figure.facecolor'] = (1., 0., 0., 0.3)  # Debugging
matplotlib.rcParams['figure.facecolor'] = (1., 0., 0., 0.)


In [4]:
# Integrity check
check_ct_edge_specificity()  # Check for duplicate edges with different attentions

100%|███████████████████████████████████████████████████████████████████| 1022/1022 [00:15<00:00, 64.77it/s]


# Metadata

In [5]:
# Load metadata
meta = get_meta()

# Subject preview
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS information available
        assert row['BRAAK_AD'] in (6,) and row['CERAD'] in (4,) and row['CDRScore'] in (3,)
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}, CERAD {row["CERAD"]}, CDR {row["CDRScore"]}, {row["Dx"]}')
filtered = np.sort(filtered)
for i in range(len(filtered)):
    # print(filtered[i])
    pass


# Attention Stack

In [6]:
fname = './attentions.pkl'
if os.path.isfile(fname):
    # Load data
    with open('./attentions.pkl', 'rb') as f:
        all_data = pickle.load(f)
    attention_stack, all_edges, columns, subject_ids = all_data['data'], all_data['edges'], all_data['heads'], all_data['subject_ids']

else:
    # Parameters
    # Scaled probably shouldn't be used, but better for visualization
    # until results are more even
    columns = get_attention_columns(scaled=False)
    subject_ids = meta['SubID'].to_numpy()

    # Load graphs
    graphs, subject_ids = load_many_graphs(subject_ids, column=columns)
    # graphs = [compute_graph(g) for g in graphs]

    # # Get attentions
    # df = {}
    # for column in get_attention_columns():
    #     attention, _ = compute_edge_summary(graphs, subject_ids=subject_ids)
    #     attention = attention.set_index('Edge')
    #     df[column] = attention.var(axis=1)


    # Set indices to edges and clean
    print('Fixing indices...')
    for i in tqdm(range(len(graphs))):
        graphs[i].index = graphs[i].apply(lambda r: get_edge_string([r['TF'], r['TG']]), axis=1)
        graphs[i] = graphs[i].drop(columns=['TF', 'TG'])
        # Remove duplicates
        graphs[i] = graphs[i][~graphs[i].index.duplicated(keep='first')]

    # Get all unique edges
    print('Getting unique edges...')
    all_edges = np.unique(sum([list(g.index) for g in graphs], []))


    # Standardize index order
    print('Standardizing indices...')
    for i in tqdm(range(len(graphs))):
        # Add missing indices and order based on `all_edges`
        # to_add = [edge for edge in all_edges if edge not in list(graphs[i].index)]  # SLOW
        to_add = list(set(all_edges) - set(graphs[i].index))

        # Empty rows
        new_rows = pd.DataFrame(
            [[np.nan]*len(graphs[i].columns)]*len(to_add),
            columns=graphs[i].columns,
        ).set_index(pd.Series(to_add))
        # Native concat
        graphs[i] = pd.concat([graphs[i], new_rows]).loc[all_edges]

    # Convert to numpy
    graphs = [g.to_numpy() for g in graphs]
    attention_stack = np.stack(graphs, axis=-1)
    # attention_stack.shape = (Edge, Head, Subject)
    # attention_stack.shape = (all_edges, columns, subject_ids)

    # Save all data
    all_data = {'data': attention_stack, 'edges': all_edges, 'heads': columns, 'subject_ids': subject_ids}
    # np.savez('attentions.npz', **all_data)
    with open(fname, 'wb') as f:
        pickle.dump(
            all_data,
            f,
            protocol=pickle.HIGHEST_PROTOCOL,
        )


In [7]:
# Additional useful parameters
self_loops = [split_edge_string(s)[0] == split_edge_string(s)[1] for s in all_edges]
self_loops = np.array(self_loops)
# Remove self loops
all_edges = all_edges[~self_loops]
attention_stack = attention_stack[~self_loops]


# Global Parameters

In [9]:
# Parameters
print(f'\nAvailable attention columns: {get_attention_columns()}')
column_ad = get_attention_columns()[0]
column_scz = get_attention_columns()[2]
column_data = get_attention_columns()[4]
synthetic_nodes_of_interest = ['OPC', 'Micro', 'Oligo']



Available attention columns: ['AD_imp_1', 'AD_imp_2', 'SCZ_imp_1', 'SCZ_imp_2', 'data_imp_1', 'data_imp_2', 'data_imp_3', 'data_imp_4']


# Intra-Contrast Comparisons

In [10]:
# Figure parameters
param = {
    'subjects': ['M31969', 'M20337'],
    'columns': [column_data, column_ad, column_scz],
    'column_names': ['Data-Driven', 'AD-Prior', 'SCZ-Prior'],
    'column_groups': [get_attention_columns()[4:8], get_attention_columns()[:2], get_attention_columns()[2:4]],
    'column_group_names': ['Data Prioritization', 'AD Prioritization', 'SCZ Prioritization'],
    'ancestries': meta.groupby('Ethnicity').count()['SubID'].sort_values().index[::-1].to_list()[:3] + ['all'],
    'contrast': 'c15x',
}

# Generate palette
palette = plt.rcParams['axes.prop_cycle'].by_key()['color']
param['palette'] = {sid: rgba_to_hex(palette[i]) for i, sid in enumerate(param['subjects'])}

# Preview subjects
for sid in param['subjects']:
    row = meta.loc[meta['SubID']==sid].iloc[0]
    # print(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}, CERAD {row["CERAD"]}, CDR {row["CDRScore"]}, {row["Dx"]}')


In [11]:
# Subplot layout (doesn't work well with constrained layout)
# NOTE: This cannot be used, as constrained layout has glitches
# (see https://github.com/matplotlib/matplotlib/issues/23290)
# with uneven mosaics
# fig, axs = get_mosaic(shape, figsize=(int((3/2) * shape_array.shape[1]), int((3/2) * shape_array.shape[0])), constrained_layout=False)

# Subfigure layout (longer)
# NOTE: Constrained layout will fail for all
# subplots if a single one is not able to scale.
# Also, sometimes leaving a subfigure blank will
# cause it to fail, especially if on an edge.
# It is VERY finnicky.
# SOLUTION: Save again using `fig.savefig(...)`
# and it will run without warning.  Then, you
# can visually inspect for scaling issues.
# fig, axs = create_subfigure_mosaic(shape_array)
# fig.set_constrained_layout_pads(w_pad=0, h_pad=0, wspace=.4, hspace=.4)  # *_pad is pad for figs (including subfigs), *_space is pad between subplots


## Edge Prioritization and Cross-Ancestry Enrichment - Figure 5cd

In [13]:
shape = """
    NNNNNNNNNNN
    NNNNNNNNNNN
    NNNNNNNNNNN
    NNNNNNNNNNN
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
    RRRRRRRRRRR
"""
fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

axs_lab = (len(param['ancestries']) - 1) * ['None'] + ['N']
# axs_lab = ['K', 'L', 'M', 'N']
print(f'\nEdge Discovery Enrichment ({", ".join(axs_lab)})')
for ancestry, ax in zip(param['ancestries'], [axs[lab] if lab in axs else None for lab in axs_lab]):
    # Filter to ancestry
    anc_data = all_data.copy()
    if ancestry != 'all':
        sub_ids = meta.loc[meta['Ethnicity'] == ancestry, 'SubID'].to_list()
        mask = [sid in sub_ids for sid in anc_data['subject_ids']]
        anc_data['data'] = anc_data['data'][:, :, mask]
        anc_data['subject_ids'] = np.array(anc_data['subject_ids'])[mask]

    # Run
    temp = plot_edge_discovery_enrichment(
        **anc_data,
        column=param['columns'][0],
        range_colors=[rgb_to_float(hex_to_rgb('#7aa457')), rgb_to_float(hex_to_rgb('#a46cb7')), rgb_to_float(hex_to_rgb('#cb6a49'))],
        ax=ax,
        postfix=f'{ancestry}_{param["columns"][0]}',
        gene_max_num=300,
        threshold=95,
        clamp_min=4,
        skip_plot=(ax is None),
        verbose=True)
    if ax is not None:
        ax.set_xlabel(f'High-Scoring Edges ({param["column_names"][0]})')
        ylabel = 'Frequency'
        if ancestry != 'all': ylabel += f' ({ancestry})'
        ax.set_ylabel(ylabel)
    # MANUAL PROCESSING
    # Run the output '../plots/genes_<column>.csv' from above on Metascape as multiple gene list and perform
    # enrichment.  From the all-in-one ZIP file, save the file from Enrichment_GO/GO_membership.csv as '../plots/go_<column>.csv'
    # and rerun.

axs_lab = ['R']
print(f'\nAncestry Enrichment Comparison ({", ".join(axs_lab)})')
postfixes = [f'{ancestry}_{param["columns"][0]}' for ancestry in param['ancestries']]
enrichments = plot_cross_enrichment(postfixes, names=param['ancestries'], ax=axs[axs_lab[0]], excluded_subgroups=['all'])

# Place labels
offset = plot_labels(axs, shape=shape)

# Save figure
print('\nSaving Figure...')
fig.savefig(f'../plots/figure_5_main.pdf', bbox_inches='tight', pad_inches=1, format='pdf', transparent=True, backend='cairo')



Edge Discovery Enrichment (None, None, None, N)
Prioritization Ranges [[14.0, 22.0], [22.0, 49.0], [49.0, 680.0]]
Filtered 165669 edges of 204507 total from histogram
Prioritization Ranges [[9.0, 13.0], [13.0, 25.0], [25.0, 104.0]]
Filtered 52826 edges of 61429 total from histogram
Prioritization Ranges [[8.0, 11.0], [11.0, 21.0], [21.0, 88.0]]
Filtered 46979 edges of 53899 total from histogram
Prioritization Ranges [[15.0, 23.0], [23.0, 51.0], [51.0, 875.0]]
Filtered 195859 edges of 242486 total from histogram

Ancestry Enrichment Comparison (R)
Index(['AFR_data_imp_1', 'AMR_data_imp_1', 'EUR_data_imp_1', 'all_data_imp_1'], dtype='object', name='Ancestry')
Index(['EUR', 'AFR', 'AMR', 'all'], dtype='object')
Index(['EUR', 'AFR', 'AMR'], dtype='object')

Saving Figure...


## PRS Analyses - Figure 6a

In [15]:
# SCZ
# Data
# AD
# SCZ
shape = """
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
    SSSSSSSSSSSSSSSUUUUUUUUUUUUUUU
"""
fig, axs = create_subfigure_mosaic(shape_array_from_shape(shape))

# Plot all panels
axs_lab = ['S', 'U']
print(f'\nPRS Analysis ({", ".join(axs_lab)})')
# Takes around an hour for each loop with no subsampling (on first run)
for fname, head_prefix, ylabel, prs_col, ax_idx in zip(
    ('ad_prs_df.csv', 'scz_prs_df.csv'),
    ('_'.join(column_ad.split('_')[:-1]), '_'.join(column_scz.split('_')[:-1])),
    ('AD Importance Score', 'SCZ Importance Score'),
    ('prs_scaled_AD_Bellenguez', 'prs_scaled_SCZ.3.5_MVP'),
    axs_lab
):
    df = pd.read_csv(fname, index_col=0) if os.path.isfile(fname) else None
    covariates = get_genotype_meta()[['SubID', 'imp_sex_score'] + [f'imp_anc_PC{i}' for i in range(1, 7)] + [f'imp_anc_{anc}' for anc in ('AFR', 'AMR', 'EAS', 'EUR')]]
    df, prs_df, axs[ax_idx] = plot_prs_correlation(
        meta, **all_data, ax=axs[ax_idx],
        df=df, num_targets=5, ylabel=ylabel, max_scale=False,
        head_prefix=head_prefix, prs_col=prs_col,
        covariates=covariates, subsample=1)
    if not os.path.isfile(fname): df.to_csv(fname)

# Place labels
offset = plot_labels(axs, shape=shape)

# Save figure
print('\nSaving Figure...')
fig.savefig(f'../plots/figure_6_prs.pdf', bbox_inches='tight', pad_inches=1, format='pdf', transparent=True, backend='cairo')


PRS Analysis (S, U)

Saving Figure...
